# Percepção Computacional - Gunshot Audio Dataset

Dataset: [Gunshot Audio Dataset](https://www.kaggle.com/datasets/emrahaydemr/gunshot-audio-dataset)

**Antes de rodar a célula de download:** configure as credenciais do Kaggle (uma vez por máquina).

1. Em [Kaggle → Account → API](https://www.kaggle.com/settings), clique em "Create Legacy API Key" que será baixado um kaggle.json, mova-o para esse diretório.

In [31]:
pip install kaggle

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [32]:
import json
import os
from pathlib import Path

raiz = Path(".").resolve()
with open(raiz / "kaggle.json", encoding="utf-8") as f:
    cred = json.load(f)
os.environ["KAGGLE_USERNAME"] = cred["username"]
os.environ["KAGGLE_KEY"] = cred["key"]

from kaggle.api.kaggle_api_extended import KaggleApi

In [33]:
api = KaggleApi()
api.authenticate()

path = Path("data")
path.mkdir(exist_ok=True)

api.dataset_download_files(
    "emrahaydemr/gunshot-audio-dataset",
    path=str(path),
    unzip=True,
)
print("Dataset baixado em:", path.resolve())

Dataset URL: https://www.kaggle.com/datasets/emrahaydemr/gunshot-audio-dataset
Dataset baixado em: E:\percepcao-computacional\data
